# Import dependencies

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, SpatialDropout2D
from keras.optimizers import RMSprop
from keras.preprocessing import image 
from keras import backend as K
import os
import numpy as np
from keras.applications.resnet50 import preprocess_input
import PIL.ImageOps    
from keras.utils import np_utils
np.random.seed(23)

# Load the Hanacaraka Dataset
Dataset courtesy of Muhammad Soleh, HPC Lab 1231 Fasilkom UI

In [ ]:
folder = 'hanacaraka/'

# Settings
img_rows, img_cols = 60, 78         
nb_classes = 20
number_of_data = 2000

X_train = []
Y_train = []

for cl, fld in enumerate(os.listdir(folder)):
        for f in os.listdir(folder + fld):
                img = image.load_img(folder + fld + '/' + f, grayscale=True, 
                                     target_size=(img_rows, img_cols))
                img = PIL.ImageOps.invert(img)
                img = image.img_to_array(img)
                X_train.append(img.reshape((img_rows, img_cols, 1)))
                Y_train.append(cl)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

shuffled_indices = np.random.permutation(X_train.shape[0])
X_train = X_train[shuffled_indices] 
Y_train = Y_train[shuffled_indices] 
X_test = X_train[X_train.shape[0]//2:]
X_train = X_train[:X_train.shape[0]//2]

Y_test = Y_train[Y_train.shape[0]//2:]
Y_train = Y_train[:Y_train.shape[0]//2]

print('X_train original shape:', X_train.shape)

if K.image_dim_ordering() == 'th':
    # For Theano backend
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # For TensorFlow backend
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# Split train test
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

test_data = 1000

X_train = X_train[:number_of_data]
Y_train = Y_train[:number_of_data]
X_test = X_test[:test_data]
Y_test = Y_test[:test_data]

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
    

# Visualize Hanacaraka dataset
import matplotlib.pyplot as plt
%matplotlib inline

n = 10  
plt.figure(figsize=(10, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_train[i].reshape(img_rows, img_cols))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Convolutional Neural Network

## Build

In [ ]:
# Settings
pool_size = (3, 3)                  # size of pooling area for max pooling
prob_drop_conv = 0.1              # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

# Convolutional model
model = Sequential()

# conv1 layer
model.add(Conv2D(8, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(SpatialDropout2D(prob_drop_conv))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
model.add(Dense(300, activation='relu'))
model.add(Dropout(prob_drop_hidden))

# fc2 layer
model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Train

In [ ]:
# Settings
batch_size = 128
epochs = 10

# Train
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)

## Evaluate

In [ ]:
evaluation = model.evaluate(X_test, Y_test, batch_size=256, verbose=1)
print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))